In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import glob
import pickle
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from models import CNN_LSTM, SepCNN_LSTM, ConvGRU_LSTM, RandomForestBaseline, LassoModel
import joblib

In [3]:
seed = 42

# Set seed for NumPy
np.random.seed(seed)

# Set seed for Python's built-in random module
random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for Torch's CUDA operations if GPU is used
# if torch.cuda.is_available():
#     torch.backends.cudnn.deterministic = True
#     torch.cuda.manual_seed(seed)

In [4]:
# # Load a sample of the data
sample_data = np.load('./data/PROCESSED_III/2018_13_155.npy')  

# # Check the shape of the sample data
print("Shape of sample data:", sample_data.shape)

Shape of sample data: (38, 1, 128, 9)


In [5]:
# Define generator function
def generator(IDs, yields, batch_size, cutoff=None):
    def load_data(ID):
        try:
            data = np.load('./data/PROCESSED_III/' + ID + '.npy')
            return data, True
        except Exception as e:
            return None, False

    batches = 0

    while True:
        batch_features = np.zeros((batch_size, 38, 1, 128, 9)) if cutoff is None else np.zeros((batch_size, cutoff, 1, 128, 9))
        batch_yields = np.zeros(batch_size)

        if batches == len(IDs) // batch_size:
            batches = 0
            yield None, None

        for i in range(batch_size):
            while True:
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                data, success = load_data(ID)
                if success:
                    break

            if data is not None:
                if cutoff is not None:
                    if not np.isnan(data).any():
                        batch_features[i, :, :, :, :] = data[:cutoff, :, :, :]
                        batch_yields[i] = yields[ID]
                else:
                    batch_features[i, :, :, :, :] = data
                    batch_yields[i] = yields[ID]
                

        batches += 1

        yield torch.tensor(batch_features, dtype=torch.float32, device='cuda'), torch.tensor(batch_yields, dtype=torch.float32, device='cuda')


In [6]:
# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))

# Generators
training_generator = generator(list(yields['train'].keys()), yields['train'], 16)
# validation_generator = generator(list(yields['validation'].keys()), yields['validation'], 16)

validation_generator = generator(list(yields['train'].keys()), yields['train'], 16)


In [14]:
n_estimators_values = [50, 100, 150]  
max_depth_values = [None, 10, 20]

# Initialize variables to store best parameters and corresponding MSE
best_params = None
best_mse = float('inf')  

for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        random_forest_model = RandomForestBaseline(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

        # Fit the model to training data
        X_train, y_train = next(training_generator)
        random_forest_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

        # Make predictions on test data
        X_test, y_test = next(validation_generator)
        predictions = random_forest_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))

        # Evaluate the model
        mse = random_forest_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())

        # Print MSE for current parameter combination
        print(f"Parameters: n_estimators={n_estimators}, max_depth={max_depth} MSE: {mse}")

        # Check if current combination improves performance
        if mse < best_mse:
            print('save best model')
            joblib.dump(random_forest_model, 'random_forest_best_model.pkl')            
            best_mse = mse
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

# Print best parameters and corresponding MSE
print("\nBest Parameters:")
print(best_params)
print("Best Mean Squared Error:", best_mse)

Parameters: n_estimators=50, max_depth=None MSE: 663.8719968404296
save best model
Parameters: n_estimators=50, max_depth=10 MSE: 1870.8201142284768
Parameters: n_estimators=50, max_depth=20 MSE: 2161.919268979579
Parameters: n_estimators=100, max_depth=None MSE: 1222.9131032222751
Parameters: n_estimators=100, max_depth=10 MSE: 1715.5148969166114
Parameters: n_estimators=100, max_depth=20 MSE: 1348.1380253248612
Parameters: n_estimators=150, max_depth=None MSE: 1065.5021248333226
Parameters: n_estimators=150, max_depth=10 MSE: 961.7749221209135
Parameters: n_estimators=150, max_depth=20 MSE: 542.6614050149682
save best model

Best Parameters:
{'n_estimators': 150, 'max_depth': 20}
Best Mean Squared Error: 542.6614050149682


In [15]:
alpha_values = [0.3, 0.4, 0.5]

best_alpha = None
best_mse = float('inf') 

# Iterate over alpha values
for alpha in alpha_values:
    # Initialize Lasso model with current alpha value
    lasso_model = LassoModel(alpha=alpha, random_state=42)

    # Fit the model to training data
    X_train, y_train = next(training_generator)
    lasso_model.fit(X_train.cpu().reshape(X_train.shape[0], -1), y_train.cpu())

    # Make predictions on test data
    X_test, y_test = next(validation_generator)
    predictions = lasso_model.predict(X_test.cpu().reshape(X_test.shape[0], -1))

    # Evaluate the model
    mse = lasso_model.evaluate(X_test.cpu().reshape(X_test.cpu().shape[0], -1), y_test.cpu())

    print(f"Alpha: {alpha}, MSE: {mse}")

    # Check if current alpha improves performance
    if mse < best_mse:
        print('save best model')
        joblib.dump(lasso_model, 'lasso_best_model.pkl')   
        best_mse = mse
        best_alpha = alpha

# Print best alpha and corresponding MSE
print("\nBest Alpha:", best_alpha)
print("Best Mean Squared Error:", best_mse)

Alpha: 0.3, MSE: 1232.5351081241095
save best model
Alpha: 0.4, MSE: 956.7630826949603
save best model
Alpha: 0.5, MSE: 1127.524033871964

Best Alpha: 0.4
Best Mean Squared Error: 956.7630826949603


In [16]:
model_functions = {
    'CNN_LSTM': CNN_LSTM,
    'SepCNN_LSTM': SepCNN_LSTM,
    'ConvGRU_LSTM': ConvGRU_LSTM,
}


epochs = 100

for model_name, model_function in model_functions.items():
    model = model_function(dimensions=[38, 1, 128, 9])
    model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    best_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch_data, batch_labels in tqdm.tqdm(training_generator, desc=f"Epoch {epoch+1}/{epochs}"):
            if batch_data is None:
                break

            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for val_data, val_labels in validation_generator:
                if val_data is None:
                    break
                val_outputs = model(val_data)
                val_loss = criterion(val_outputs, val_labels.unsqueeze(1))
                val_losses.append(val_loss.item())

        current_loss = np.mean(val_losses)
        if current_loss < best_loss:
            torch.save(model, f'{model_name}_best.pt')
            best_loss = current_loss
            
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {current_loss:.4f}")

Epoch 1/100: 0it [00:00, ?it/s]/home/kaywee/Documents/wee's things/cds-corn-prediction/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/100: 377it [00:12, 29.65it/s]


Epoch 1/100, Train Loss: 22312.9983, Val Loss: 18670.1784


Epoch 2/100: 389it [00:11, 32.81it/s]


Epoch 2/100, Train Loss: 12676.2649, Val Loss: 8271.3418


Epoch 3/100: 389it [00:11, 32.54it/s]


...

Epoch 98/100, Train Loss: 526.9250, Val Loss: 128.9720


Epoch 99/100: 389it [00:29, 13.33it/s]


Epoch 99/100, Train Loss: 543.2133, Val Loss: 71.1486


Epoch 100/100: 389it [00:29, 13.33it/s]


Epoch 100/100, Train Loss: 525.8892, Val Loss: 58.5037


In [10]:
trained_model = torch.load('./models/CNN_LSTM_best.pt')

trained_model.eval()
res = trained_model(torch.tensor(np.array([sample_data]), dtype=torch.float32, device='cuda'))
target = yields['train']['2018_13_155']
print(res, target)

/home/kaywee/Documents/wee's things/cds-corn-prediction/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv2d(input, weight, bias, self.stride,


OutOfMemoryError: CUDA out of memory. Tried to allocate 13.50 GiB. GPU 0 has a total capacity of 10.75 GiB of which 8.64 GiB is free. Including non-PyTorch memory, this process has 1.30 GiB memory in use. Of the allocated memory 1.13 GiB is allocated by PyTorch, and 1.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trained_model = torch.load('./models/CNN_LSTM_best.pt')

test_gen = generator(list(yields['validation'].keys()), yields['validation'], len(yields['validation']))
X_test, y_test = next(test_gen)
